In [25]:
import os
from dotenv import load_dotenv

load_dotenv("../config/.env")
# os.environ["LANGSMITH_TRACING"] = "true"

True

In [26]:
from langchain.chat_models import init_chat_model

model = init_chat_model("deepseek-chat", model_provider="deepseek")

In [27]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="你好，我是Frank！")])

AIMessage(content='你好，Frank！很高兴认识你。我是DeepSeek AI助手，随时为你提供帮助。无论是问题解答、信息查询，还是闲聊交流，我都在这里哦！有什么可以帮你的吗？😊', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 9, 'total_tokens': 53, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 9}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_08f168e49b_prod0820_fp8_kvcache', 'id': '8f8fa6f5-40d2-424c-b804-01e7c6bad771', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--abefebf5-15d4-46fd-acb9-dc6ce87262a9-0', usage_metadata={'input_tokens': 9, 'output_tokens': 44, 'total_tokens': 53, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})

In [28]:
model.invoke([HumanMessage(content="我叫什么名字？")])

AIMessage(content='我不知道你的名字，但如果你告诉我，我会记住并在以后的对话中称呼你！😊', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 8, 'total_tokens': 27, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 8}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_08f168e49b_prod0820_fp8_kvcache', 'id': '63f8faab-4dbc-4ac7-bc90-729429983f18', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--3c5bbbe4-3773-492b-9a5e-a76f49b9b726-0', usage_metadata={'input_tokens': 8, 'output_tokens': 19, 'total_tokens': 27, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})

In [29]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}

workflow = StateGraph(state_schema=MessagesState)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

config = {"configurable": {"thread_id": "thread_1"}}
query = "你好，我是Frank！"
input_message = [HumanMessage(query)]
output = app.invoke({"messages": input_message}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

你好，Frank！很高兴认识你。我是DeepSeek-V3，随时为你提供帮助。有什么我可以为你做的吗？😊


In [30]:
query = "我叫什么名字？"
imput_message = [HumanMessage(query)]
output = app.invoke({"messages": input_message}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

你好，Frank！很高兴认识你！😊  
我是DeepSeek-V3，你的智能助手，随时准备为你解答问题或提供帮助。有什么我可以为你做的吗？


In [31]:
query = "我叫什么名字？"
input_message = [HumanMessage(query)]
config = {"configurable": {"thread_id": "thread_2"}}
output = app.invoke({"messages": input_message}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

我不知道你的名字，但如果你告诉我，我会记住并在以后的对话中称呼你！😊
